In [1]:
from neo4j import GraphDatabase,basic_auth
from random import randint,choice
import datetime

In [2]:
#Create Connection to Neo4j

conn = GraphDatabase.driver(uri = "bolt://localhost:7687",auth=("vishal",'123456'))


In [3]:
conn.address

IPv4Address(('100.26.227.192', 38697))

In [4]:
conn.session()

In [5]:
conn.verify_connectivity()

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: ExperimentalWarning: The configuration may change in the future.
  """Entry point for launching an IPython kernel.


'Neo4j/3.5.17'

In [5]:
sess = conn.session(database="customerOrder")

In [ ]:
sess.run(query="CREATE(a:Product{ProdName:'Pen'})")
sess.run("CREATE(a:Product{ProdName:'Printer'})")
sess.run("CREATE(s:Supplier{SuppName :'Reynolds'})")
sess.run("CREATE(s:Supplier{SuppName :'Brother'})")

sess.run("MATCH(a:Product{ProdName:'Pen'}) \
         MATCH(s:Supplier{SuppName :'Reynolds'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MATCH(s:Supplier{SuppName :'Brother'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MERGE(s:Supplier{SuppName :'Cannon'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MERGE(s:Supplier{SuppName :'HP'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MERGE(a:Product{ProdName:'File Cover'}) \
         MERGE(s:Supplier{SuppName :'up&up'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MERGE(a:Product{ProdName:'Envelopes'}) \
         MERGE(s:Supplier{SuppName :'Prince'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

sess.run("MERGE(a:Product{ProdName:'Markers'}) \
         MERGE(s:Supplier{SuppName :'Reynolds'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")


In [10]:
sess.run("Create(c:Category{CategoryNm : 'Stationary'})")
sess.run("MATCH(c:Category) \
          MATCH (p:Product)\
          Create (p)-[:HAS_CATEGORY]->(c)\
          "
        )

In [11]:
prod_list = ['Printer','Pen','Markers','File Cover','Notebooks','Envelopes']
cust_list =['WF','BoA','JPM','Citi','USB','5/3rd','FirstCitizen','Walmart','Kroger','HT','CTS','TIAA']

In [12]:
sess.run("CREATE(a:Customer{CompanyName:'WF'})")
sess.run("CREATE(a:Customer{CompanyName:'BoA'})")
sess.run("CREATE(a:Customer{CompanyName:'JPM'})")
sess.run("CREATE(a:Customer{CompanyName:'Citi'})")
sess.run("CREATE(a:Customer{CompanyName:'USB'})")
sess.run("CREATE(a:Customer{CompanyName:'5/3rd'})")
sess.run("CREATE(a:Customer{CompanyName:'FirstCitizen'})")
sess.run("CREATE(a:Customer{CompanyName:'Walmart'})")
sess.run("CREATE(a:Customer{CompanyName:'Kroger'})")
sess.run("CREATE(a:Customer{CompanyName:'HT'})")
sess.run("CREATE(a:Customer{CompanyName:'CTS'})")
sess.run("CREATE(a:Customer{CompanyName:'TIAA'})")

In [13]:
%%time
for order in range(100001,100101):
    #print('Order_id : ' + str(order))
    prod=choice(prod_list)
    #print(prod)
    
    start_date = datetime.date(2020, 1, 1)
    random_days = randint(1,364)
    Order_date = start_date + datetime.timedelta(days=random_days)
    #print(Order_date)
    
    qty=randint(1,100)
    #print(qty)
    
    CompanyName=choice(cust_list)
    
    
    line_item = randint(1,6)
    if line_item==1:
        #Insert Data into Graph
        sess.run("MATCH (p:Product{ProdName : $prod})\
                  MATCH (c:Customer{CompanyName : $CompanyName})\
                  MERGE(o:Order{OrderId:$order,OrderDt:$Order_date }) \
                  MERGE (c)-[:PLACED_ORDER]->(o)-[:HAS_PROD{Qty:$qty}]->(p)\
                 ",
                 prod=prod,
                 order=order,
                 Order_date=Order_date,
                 CompanyName=CompanyName,
                 qty=qty
            )
        print(str(order) +' '+CompanyName +' '+prod+' '+str(Order_date)+' '+str(qty))
    else:
        #Insert Data into Graph
        sess.run("MATCH (c:Customer{CompanyName : $CompanyName})\
                  MERGE(o:Order{OrderId:$order,OrderDt:$Order_date }) \
                  MERGE (c)-[:PLACED_ORDER]->(o)\
                 ",                 
                 order=order,
                 Order_date=Order_date,
                 CompanyName=CompanyName
                )
        prod_tmp_lst =[]
        for i in range(1,line_item+1):
            prod=choice(prod_list)
            if prod in prod_tmp_lst:
                continue
            prod_tmp_lst.append(prod)
            qty=randint(1,100)
            #Insert Data into Graph
            sess.run("MATCH (p:Product{ProdName : $prod})\
                      MATCH (o:Order{OrderId:$order }) \
                      MERGE (o)-[:HAS_PROD{Qty:$qty}]->(p)\
                     ",
                     prod=prod,
                     order=order,
                     qty=qty
                )
            print('ELSE : Line_Item : '+str(i)+' '+str(order) +' '+CompanyName+' '+str(Order_date) +' '+prod+' '+str(qty))
    print('\n')

100001 JPM Pen 2020-05-06 59


ELSE : Line_Item : 1 100002 HT 2020-06-21 Markers 43
ELSE : Line_Item : 3 100002 HT 2020-06-21 Pen 40
ELSE : Line_Item : 4 100002 HT 2020-06-21 Envelopes 88
ELSE : Line_Item : 5 100002 HT 2020-06-21 Printer 19


ELSE : Line_Item : 1 100003 Kroger 2020-03-21 Notebooks 53
ELSE : Line_Item : 2 100003 Kroger 2020-03-21 Envelopes 46
ELSE : Line_Item : 3 100003 Kroger 2020-03-21 File Cover 99
ELSE : Line_Item : 4 100003 Kroger 2020-03-21 Markers 97


ELSE : Line_Item : 1 100004 WF 2020-07-17 Markers 44
ELSE : Line_Item : 2 100004 WF 2020-07-17 File Cover 77
ELSE : Line_Item : 3 100004 WF 2020-07-17 Notebooks 78
ELSE : Line_Item : 4 100004 WF 2020-07-17 Pen 77


ELSE : Line_Item : 1 100005 Walmart 2020-06-04 Pen 30
ELSE : Line_Item : 2 100005 Walmart 2020-06-04 Markers 5
ELSE : Line_Item : 5 100005 Walmart 2020-06-04 Envelopes 73
ELSE : Line_Item : 6 100005 Walmart 2020-06-04 Notebooks 63


100006 WF Envelopes 2020-10-26 79


ELSE : Line_Item : 1 100007 Citi 202

ELSE : Line_Item : 2 100053 JPM 2020-11-21 Pen 14


ELSE : Line_Item : 1 100054 FirstCitizen 2020-09-01 Notebooks 58
ELSE : Line_Item : 3 100054 FirstCitizen 2020-09-01 Printer 90
ELSE : Line_Item : 4 100054 FirstCitizen 2020-09-01 Markers 84


ELSE : Line_Item : 1 100055 FirstCitizen 2020-07-26 File Cover 11
ELSE : Line_Item : 2 100055 FirstCitizen 2020-07-26 Envelopes 1
ELSE : Line_Item : 3 100055 FirstCitizen 2020-07-26 Pen 89


ELSE : Line_Item : 1 100056 USB 2020-06-15 Notebooks 5
ELSE : Line_Item : 2 100056 USB 2020-06-15 Printer 64
ELSE : Line_Item : 3 100056 USB 2020-06-15 Envelopes 72


ELSE : Line_Item : 1 100057 BoA 2020-03-25 Printer 53
ELSE : Line_Item : 2 100057 BoA 2020-03-25 Notebooks 45
ELSE : Line_Item : 4 100057 BoA 2020-03-25 Envelopes 68


ELSE : Line_Item : 1 100058 Citi 2020-11-06 File Cover 79
ELSE : Line_Item : 2 100058 Citi 2020-11-06 Notebooks 69
ELSE : Line_Item : 3 100058 Citi 2020-11-06 Markers 46


ELSE : Line_Item : 1 100059 BoA 2020-08-29 Notebooks 100
